In [1]:
import h5py
from glob import glob
import sys, scipy
from scipy.stats import chi2

import matplotlib.pyplot as plt
import numpy as np

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [3]:
pf_features = ['Energy', 'Px', 'Py', 'Pz', 'Pt', 'Eta', 'Phi', 
                'vtxX', 'vtxY', 'vtxZ','ChPFIso', 'GammaPFIso', 'NeuPFIso',
                'isChHad', 'isNeuHad', 'isGamma', 'isEle',  'isMu', 
                #'Charge'
               ]
hlf_features = ['HT', 'MET', 'phiMET', 'MT', 'nJets', 'nBjets',
                'LepPt', 'LepEta', 'LepPhi', 'LepIsoCh',
                'LepIsoGamma', 'LepIsoNeu', 'LepCharge', 'LepIsEle'
               ]

# Start the VAE declaration

In [ ]:
batch_size = 100
original_dim = len(hlf_features)
latent_dim = 2
intermediate_dim = 40
epsilon_std = 1.0

In [ ]:
x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='tanh')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [ ]:
decoder_h = Dense(intermediate_dim, activation='tanh')
decoder_mean = Dense(original_dim)
decoder_log_var = Dense(original_dim)

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
x_decoded_log_var = decoder_log_var(h_decoded)

Custom loss layer

In [ ]:
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean, x_decoded_log_var):
        norm_x = K.tf.divide(x-x_decoded_mean, K.exp(x_decoded_log_var))
        single_L = 2*x_decoded_log_var + 0.5*K.square(norm_x)
        nll_loss = K.sum(single_L, axis=-1)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(nll_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        x_decoded_log_var = inputs[2]
        loss = self.vae_loss(x, x_decoded_mean, x_decoded_log_var)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x
    
y = CustomVariationalLayer()([x, x_decoded_mean, x_decoded_log_var])

In [ ]:
vae = Model(x, y)
vae.compile(optimizer='adam', loss=None)

# Training

In [ ]:
data_folder = '~/cernbox/AnomalyDetection/data/MaxLepDeltaR_des/'

train_list = glob(data_folder +'train/*.h5')
hlf_train = np.zeros((0,14))
label_train = np.zeros((0,))
for fname in train_list:
    f = h5py.File(fname)
    hlf_train = np.concatenate((hlf_train, np.array(f['HLF'])))
    label_train = np.concatenate((label_train, np.array(f['Labels'])[:,1] + 2*np.array(f['Labels'])[:,2]))

cut = label_train==0
x_train = hlf_train[cut]
l_train = label_train[cut]
print 'Train data shape: ', x_train.shape



val_list = glob(data_folder +'val/*.h5')
hlf_val = np.zeros((0,14))
label_val = np.zeros((0,))
for fname in val_list:
    f = h5py.File(fname)
    hlf_val = np.concatenate((hlf_val, np.array(f['HLF'])))
    label_val = np.concatenate((label_val, np.array(f['Labels'])[:,1] + 2*np.array(f['Labels'])[:,2]))

x_test = hlf_val #[f_val['Labels'][:,0]==1,:]
print 'Val data shape: ', x_test.shape

# print f_train.keys()

In [ ]:
fit_report = vae.fit(x_train,
        shuffle=True,
        epochs=200,
        batch_size=batch_size,
        validation_data=(x_test, None),
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, verbose=1),
            ReduceLROnPlateau(monitor='loss', factor=0.3, patience=5, verbose=1)
            ])

In [ ]:
for item in ['loss', 'val_loss']:
    plt.semilogy(fit_report.history[item][10:], label=item)
plt.xlabel('Epoch')
plt.grid()
plt.legend(loc='best')

# Build encoder and decoder

### Encoder

In [ ]:
encoder = Model(inputs=x, outputs=[z_mean, z_log_var])

In [ ]:
x_train_encoded = np.array(encoder.predict(x_train, batch_size=batch_size))
x_train_encoded.shape
plt.figure()
plt.scatter(x_train_encoded[0, :, 0], x_train_encoded[0, :, 1], c=l_train)
plt.show()

In [ ]:
x_test_encoded = np.array(encoder.predict(x_test, batch_size=batch_size))
x_test_encoded.shape
plt.figure()
plt.scatter(x_test_encoded[0, :, 0], x_test_encoded[0, :, 1], c=label_val)
plt.colorbar()
plt.show()

### Probabilistic Decoder

In [ ]:
sqrt_2pi = np.sqrt(2*np.pi)
class GaussProbabilityComputerLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(GaussProbabilityComputerLayer, self).__init__(**kwargs)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        x_decoded_log_var = inputs[2]
        norm_x = K.tf.divide(x-x_decoded_mean, K.exp(x_decoded_log_var))
        exp_part = K.exp(-0.5*K.square(norm_x))
        prob = K.tf.divide(exp_part,K.exp(x_decoded_log_var))/sqrt_2pi
        return prob

In [ ]:
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_log_var = decoder_log_var(_h_decoded)
decoder = Model(decoder_input, [_x_decoded_mean, _x_decoded_log_var])

In [ ]:
def Gaussian_np(x, mu=0, sigma=1):
    z = (x-mu)/sigma
    return np.exp(-0.5*np.square(z))/(sigma*np.sqrt(2*np.pi))

In [ ]:
np.max(np.abs(Gaussian_np(np.random.uniform(size=(400)))))

In [ ]:
x_aux = x_train[:1,:]
print 'aux:', aux
pred = encoder.predict(aux)
print 'Latent Space'
print pred[0]
print np.exp(pred[1])

z_aux = np.random.normal(loc=pred[0], scale=np.exp(pred[1]))
print z_aux

print 'Predicted x'
pred_x = decoder.predict(z)
mu = pred_x[0]
sigma = np.exp(pred_x[1])
print mu
print sigma
x_norm = (x_aux-mu)/sigma
print x_norm
pvalue = 1-np.abs(scipy.special.erf(x_norm))/np.sqrt(2)
print 'pvalues', pvalue

comb_pvalue = 1-chi2.cdf(-np.sum(np.log(pvalue)), 2*pvalue.shape[1])
print 'Event combined p-value:', comb_pvalue

In [ ]:
comb_pvalue = []
for x_aux in x_train:
    pred = encoder.predict(aux)
    z_aux = np.random.normal(loc=pred[0], scale=np.exp(pred[1]))
    pred_x = decoder.predict(z)
    mu = pred_x[0]
    sigma = np.exp(pred_x[1])
    x_norm = (x_aux-mu)/sigma
    pvalue = 1-np.abs(scipy.special.erf(x_norm))/np.sqrt(2)
    comb_pvalue.append(1-chi2.cdf(-np.sum(np.log(pvalue)), 2*pvalue.shape[1]))

In [ ]:
plt.hist(comb_pvalue, bins=20)
plt.yscale('log', nonposy='clip')

In [ ]:
_gaus_prob = GaussProbabilityComputerLayer()([x, _x_decoded_mean, _x_decoded_log_var])